In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import timedelta

In [3]:

# 读取数据
df = pd.read_csv('dataset/GOOGL_2023.csv')

In [4]:
# 数据标准化
minmax = MinMaxScaler().fit(df.iloc[:, 4:5].astype('float32'))  # Close index
df_log = minmax.transform(df.iloc[:, 4:5].astype('float32'))  # Close index
df_log = pd.DataFrame(df_log)

In [5]:
# 划分训练集和测试集
test_size = 30
simulation_size = 10
df_train = df_log.iloc[:-test_size]
df_test = df_log.iloc[-test_size:]

In [6]:
# 定义LSTM模型
class LSTM(nn.Module):
    def __init__(self, num_layers, size_layer, input_size, output_size, dropout_rate):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.size_layer = size_layer
        self.lstm = nn.LSTM(input_size, size_layer, num_layers, batch_first=True, dropout=dropout_rate)
        self.fc = nn.Linear(size_layer, output_size)

    def forward(self, x, hidden):
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])  # 取最后一个时间步的输出
        return out, hidden

    def init_hidden(self, batch_size, device):
        return (torch.zeros(self.num_layers, batch_size, self.size_layer).to(device),
                torch.zeros(self.num_layers, batch_size, self.size_layer).to(device))

In [7]:
# 定义训练和预测函数
def train_model(model, df_train, num_epochs, timestamp, criterion, optimizer, device):
    model.train()
    total_loss = []
    
    for epoch in range(num_epochs):
        hidden = model.init_hidden(1, device)  # 每次 epoch 初始化 hidden
        epoch_loss = 0.0
        
        # 使用 tqdm 进度条
        with tqdm(total=df_train.shape[0] - 1, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='step') as pbar:
            for k in range(0, df_train.shape[0] - 1, timestamp):
                optimizer.zero_grad()  # 每次优化前清空梯度
                index = min(k + timestamp, df_train.shape[0] - 1)
                batch_x = torch.FloatTensor(np.expand_dims(df_train.iloc[k:index, :].values, axis=0)).to(device)
                batch_y = torch.FloatTensor(df_train.iloc[k + 1:index + 1, :].values).to(device)

                # 前向传播
                output, hidden = model(batch_x, hidden)
                
                # 计算损失并反向传播
                loss = criterion(output, batch_y)
                
                # 反向传播并更新权重，确保 retain_graph 只在必要时使用
                loss.backward(retain_graph=False)
                optimizer.step()

                # 累积损失
                epoch_loss += loss.item()
                total_loss.append(loss.item())

                # 更新 tqdm 进度条
                pbar.set_postfix(loss=loss.item())
                pbar.update(timestamp)
        
        # 打印每个 epoch 的平均损失
        avg_epoch_loss = epoch_loss / (df_train.shape[0] // timestamp)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Avg Loss: {avg_epoch_loss:.6f}')
        
    return np.mean(total_loss)

# 在训练模型之前启用异常检测
torch.autograd.set_detect_anomaly(True)


In [8]:
def forecast(model, df_train, df_test, timestamp, future_days, device):
    model.eval()
    hidden = model.init_hidden(1, device)
    output_predict = np.zeros((df_train.shape[0] + future_days, df_train.shape[1]))
    output_predict[0] = df_train.iloc[0]
    
    for k in range(0, df_train.shape[0] - 1, timestamp):
        batch_x = torch.FloatTensor(np.expand_dims(df_train.iloc[k:k + timestamp].values, axis=0)).to(device)
        with torch.no_grad():
            out_logits, hidden = model(batch_x, hidden)
        output_predict[k + 1:k + timestamp + 1] = out_logits.cpu().numpy()
    
    # 预测未来
    for i in range(future_days):
        input_seq = torch.FloatTensor(np.expand_dims(output_predict[-timestamp:], axis=0)).to(device)
        with torch.no_grad():
            out_logits, hidden = model(input_seq, hidden)
        output_predict[-future_days + i] = out_logits.cpu().numpy()

    output_predict = minmax.inverse_transform(output_predict)
    return output_predict[-test_size:]

In [9]:
# 初始化模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_layers = 1
size_layer = 128
timestamp = 5
num_epochs = 300
dropout_rate = 0.8
learning_rate = 0.01
output_size = 1
input_size = df_log.shape[1]

model = LSTM(num_layers, size_layer, input_size, output_size, dropout_rate).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/home/millex/stock_trading/.venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.8 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [10]:
# 训练模型并预测
train_model(model, df_train, num_epochs, timestamp, criterion, optimizer, device)
results = []

for i in range(simulation_size):
    print(f'simulation {i+1}')
    result = forecast(model, df_train, df_test, timestamp, test_size, device)
    results.append(result)

Epoch 1/300:   0%|          | 0/219 [00:00<?, ?step/s]/home/millex/stock_trading/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/300:   2%|▏         | 5/219 [00:00<00:09, 21.50step/s, loss=0.00052]/home/millex/stock_trading/.venv/lib/python3.12/site-packages/torch/autograd/graph.py:769: UserWarning: Error detected in CudnnRnnBackward0. Traceback of forward call that caused the error:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/millex/stock_trading/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/millex/stock_trading/.venv/lib/python3.12/site-p

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
# 绘制结果
plt.figure(figsize=(15, 5))
for no, r in enumerate(results):
    plt.plot(r, label=f'forecast {no + 1}')
plt.plot(df['Close'].iloc[-test_size:].values, label='true trend', c='black')
plt.legend()
plt.title(f'GOOGL_2023 average accuracy')
plt.show()